Project 1 : Multiclass Multilabel prediction For stack overflow Questions
Download data from here : https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip?dl=0

Goal : Given text for Questions , predict tags associated with them

Suggested Guidelines :

1. Pick top 10 most occurring tags and filter data for them and build your model for them

2. You can expand your model later on

3. Since there is no performance threshold and associated data given to you , you'll need to separate data in train/validation yourself

4. Before directly jumping in with LSTM ( or any other model of your choice ) , explore this text data and see if any basic data cleaning is required .

5. Usage of Answers.csv is optional

6. This is a multi class - multilabel prediction problem , keep that in mind

7. Your end solution needs to be uploaded on GitHub repo




In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
questions= pd.read_csv('/content/drive/MyDrive/Datasets/DL project Multiclass Multilabel prediction For stack overflow QuestionsPage Not completed: Project 1 : Multiclass Multilabel prediction For stack overflow Questions./Questions.csv', encoding='latin-1' )

In [ ]:
#Reads the tags file
tags = pd.read_csv('/content/drive/MyDrive/Datasets/DL project Multiclass Multilabel prediction For stack overflow QuestionsPage Not completed: Project 1 : Multiclass Multilabel prediction For stack overflow Questions./Tags.csv', encoding='latin-1')


In [ ]:
questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264216 entries, 0 to 1264215
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Id            1264216 non-null  int64  
 1   OwnerUserId   1249762 non-null  float64
 2   CreationDate  1264216 non-null  object 
 3   ClosedDate    55959 non-null    object 
 4   Score         1264216 non-null  int64  
 5   Title         1264216 non-null  object 
 6   Body          1264216 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 67.5+ MB


In [ ]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3750994 entries, 0 to 3750993
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Id      int64 
 1   Tag     object
dtypes: int64(1), object(1)
memory usage: 57.2+ MB


In [ ]:
# Merge data
data = pd.merge(questions, tags, on='Id')


In [ ]:
# Top 10 tags
top_tags = data['Tag'].value_counts().nlargest(10).index
data = data[data['Tag'].isin(top_tags)]



In [ ]:
# Text preprocessing
data['Text'] = data['Title'] + ' ' + data['Body']


In [ ]:
# Split data
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(data['Text'], data['Tag'], test_size=0.2)


In [ ]:
# One-hot encoding
#y_train_onehot = pd.get_dummies(y_train).values
#y_val_onehot = pd.get_dummies(y_val).values

# Label encoding
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_val_le = le.transform(y_val)



In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)


In [ ]:
# Naive Bayes model
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train_tfidf, y_train_le)


MultinomialNB()

In [ ]:
# Predict
y_pred = model.predict(X_val_tfidf)


In [ ]:

# Evaluate
from sklearn.metrics import accuracy_score, roc_auc_score,f1_score

accuracy = accuracy_score(y_val_le, y_pred)
f1 = f1_score(y_val_le, y_pred, average='macro')

print(f"Accuracy: {accuracy:.3f}")
print(f"F1 Score: {f1:.3f}")



Accuracy: 0.576
F1 Score: 0.517


In [ ]:
y_pred_proba = model.predict_proba(X_val_tfidf)
auc_score = roc_auc_score(y_val_le, y_pred_proba, multi_class='ovr')
print(f"AUC Score: {auc_score:.3f}")


AUC Score: 0.935


# Deep learning method

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np

Mounted at /content/drive


In [ ]:
questions_data= pd.read_csv('/content/drive/MyDrive/Datasets/DL project Multiclass Multilabel prediction For stack overflow QuestionsPage Not completed: Project 1 : Multiclass Multilabel prediction For stack overflow Questions./Questions.csv', encoding='latin-1' )

In [ ]:
#Reads the tags file
tags_data = pd.read_csv('/content/drive/MyDrive/Datasets/DL project Multiclass Multilabel prediction For stack overflow QuestionsPage Not completed: Project 1 : Multiclass Multilabel prediction For stack overflow Questions./Tags.csv', encoding='latin-1')


In [ ]:
# Merge data
data = pd.merge(questions_data, tags_data, on='Id')



In [ ]:
# Top 10 most occurring tags
top_tags = data['Tag'].value_counts().nlargest(10).index


In [ ]:
# Filter data for top 10 tags
data = data[data['Tag'].isin(top_tags)]


In [ ]:
# Text preprocessing
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from nltk.stem import WordNetLemmatizer





[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# Initialize lemmatizer and stop words only once   # less time
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Use nltk's faster regex tokenizer
    tokens = nltk.RegexpTokenizer(r'\w+').tokenize(text)

    # Lemmatize and filter stop words in one step
    tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.lower() not in stop_words]

    return ' '.join(tokens)

# Use vectorized operations instead of apply
data['Title'] = data['Title'].str.replace(r'[^a-zA-Z\s]', '', regex=True)
data['Body'] = data['Body'].str.replace(r'[^a-zA-Z\s]', '', regex=True)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


In [ ]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data[['Title', 'Body']], data['Tag'], test_size=0.2, random_state=42)



In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
# Create tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train['Title'].tolist() + X_val['Title'].tolist() + X_train['Body'].tolist() + X_val['Body'].tolist())


In [ ]:
# Convert text to sequences
train_title_sequences = tokenizer.texts_to_sequences(X_train['Title'])
train_body_sequences = tokenizer.texts_to_sequences(X_train['Body'])
val_title_sequences = tokenizer.texts_to_sequences(X_val['Title'])
val_body_sequences = tokenizer.texts_to_sequences(X_val['Body'])


In [ ]:
# Pad sequences
max_length_title = 20
max_length_body = 200
train_title_padded = pad_sequences(train_title_sequences, maxlen=max_length_title)
train_body_padded = pad_sequences(train_body_sequences, maxlen=max_length_body)
val_title_padded = pad_sequences(val_title_sequences, maxlen=max_length_title)
val_body_padded = pad_sequences(val_body_sequences, maxlen=max_length_body)


In [ ]:
labels_train_onehot = pd.get_dummies(y_train).values
labels_val_onehot = pd.get_dummies(y_val).values


In [ ]:
# Verify shapes
print("Training title shape:", train_title_padded.shape)
print("Training body shape:", train_body_padded.shape)
print("Training labels shape:", labels_train_onehot.shape)
print("Validation title shape:", val_title_padded.shape)
print("Validation body shape:", val_body_padded.shape)
print("Validation labels shape:", labels_val_onehot.shape)




Training title shape: (661391, 20)
Training body shape: (661391, 200)
Training labels shape: (661391, 10)
Validation title shape: (165348, 20)
Validation body shape: (165348, 200)
Validation labels shape: (165348, 10)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, Flatten

title_input = Input(shape=(20,), name='title')
body_input = Input(shape=(200,), name='body')

title_embedding = Embedding(10000, 64)(title_input)
body_embedding = Embedding(10000, 64)(body_input)

title_lstm = LSTM(32)(title_embedding)
body_lstm = LSTM(64)(body_embedding)

concat = Concatenate()([title_lstm, body_lstm])
dense = Dense(64, activation='relu')(concat)
output = Dense(10, activation='softmax')(dense)

model = Model(inputs=[title_input, body_input], outputs=output)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit([train_title_padded, train_body_padded], labels_train_onehot,
          validation_data=([val_title_padded, val_body_padded], labels_val_onehot),
          epochs=3, batch_size=128)



Epoch 1/3
5168/5168 ━━━━━━━━━━━━━━━━━━━━ 2040s 394ms/step - accuracy: 0.6213 - loss: 1.0618 - val_accuracy: 0.6989 - val_loss: 0.8629
Epoch 2/3
5168/5168 ━━━━━━━━━━━━━━━━━━━━ 2023s 386ms/step - accuracy: 0.7434 - loss: 0.7180 - val_accuracy: 0.7735 - val_loss: 0.6101
Epoch 3/3
5168/5168 ━━━━━━━━━━━━━━━━━━━━ 2080s 403ms/step - accuracy: 0.7859 - loss: 0.5683 - val_accuracy: 0.7810 - val_loss: 0.5802


Observations:

1. Consistent improvement in training and validation accuracy.
2. Significant decrease in training and validation loss.
3. Minimal overfitting (difference between training and validation metrics is small).

Next Steps:

1. Evaluate the model on a test dataset.
2. Consider fine-tuning hyperparameters for further improvement.
3. Deploy the model in a production environment.


In [ ]:
# Evaluate model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

loss, accuracy = model.evaluate([train_title_padded, train_body_padded], labels_val_onehot)
print(f'Validation accuracy: {accuracy:.3f}')

5168/5168 ━━━━━━━━━━━━━━━━━━━━ 202s 39ms/step - accuracy: 0.7822 - loss: 0.5758
Validation accuracy: 0.781


In [ ]:
# Make predictions
predictions = model.predict([val_title_padded, val_body_padded])
predicted_labels = np.argmax(predictions, axis=1)

5168/5168 ━━━━━━━━━━━━━━━━━━━━ 184s 36ms/step


In [ ]:
# Convert one-hot encoded val_labels to class labels
labels_val_onehot = np.argmax(labels_val_onehot, axis=1)

In [ ]:
print('Classification Report:')
print(classification_report(labels_val_onehot, predicted_labels))
print('Confusion Matrix:')
print(confusion_matrix(labels_val_onehot, predicted_labels))

Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.92      0.86     18101
           1       0.87      0.87      0.87     20459
           2       0.88      0.85      0.86      9342
           3       0.56      0.45      0.50     11668
           4       0.90      0.92      0.91      9401
           5       0.87      0.81      0.84     22817
           6       0.64      0.65      0.65     24900
           7       0.56      0.59      0.57     15796
           8       0.83      0.84      0.84     19778
           9       0.92      0.94      0.93     13086

    accuracy                           0.78    165348
   macro avg       0.78      0.78      0.78    165348
weighted avg       0.78      0.78      0.78    165348

Confusion Matrix:
[[16586   150    42    61   207   744   167    30    81    33]
 [  182 17884   576   163   130   569   482   208   143   122]
 [   85   683  7894    13   114   297    48     0    28   180]
 [  105   2